In [10]:
import pandas as pd
from tqdm.auto import tqdm
import json
from openai import OpenAI


In [7]:
with open('diabetes_data_with_ids', 'r') as file:
    diabetes_data_with_ids = json.load(file)

In [11]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [41]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='phi',
        messages=[{"role": "system", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [42]:
prompt_template = '''
You emulate a diabetes patient, formulate 3 similar questions that is simillar to the one provided below


question:{question}
answer:{answer}

Provide the output in parsable JSON without using code blocks:
return just the generated question
Don't return the main question and the answer

["question1", "question2", "question3"]
'''.strip()

In [45]:
test_data = diabetes_data_with_ids[:2]

In [46]:
results = {}
for doc in tqdm(test_data): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions

100%|██████████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:51<00:00, 55.97s/it]


In [47]:
results

{'78004d10': " Sure! Here are three additional questions that focus on healthy eating habits for people with diabetes:\nQuestion 1: What is the glycemic index and how can it help you manage your blood sugar levels when making food choices?\nAnswer: The glycemic index (GI) measures the impact of foods on our blood glucose levels. Foods with a high GI value, such as sugary snacks or refined carbohydrates, cause a rapid spike in blood sugar levels. On the other hand, low GI foods, such as whole grains and vegetables, have a slower effect on blood glucose levels. Monitoring your food choices based on their glycemic index can help you manage your blood sugar levels effectively.\nQuestion 2: How can portion control help you maintain healthy blood sugar levels?\nAnswer: Portion control involves eating appropriate amounts of food to meet your body's nutritional needs without overindulging. For people with diabetes, managing portion sizes is important as it helps to regulate blood glucose level

In [49]:
test_data[0]

{'question': 'What are some examples of calorie-free, caffeine-free liquids that individuals with diabetes can consume when sick?',
 'answer': 'Examples of calorie-free, caffeine-free liquids include water, unsweetened tea, and clear broth.',
 'id': '78004d10'}

In [51]:
import os
import json
import openai

client = openai.OpenAI(
    base_url="https://api.aimlapi.com/v1",
    api_key='ecf5e9e20fa645c9bb21b8619f584f53',
)

tools = [
  {
    "type": "function",
    "function": {
      "name": "diabetes_patient",
      "description": "Formulate questions a diabetes patients might ask",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "As a JSON parseable format e.g[question1,question2,question3]"
          }
        }
      }
    }
  }
]

messages = [
    {"role": "system", "content": "You are a helpful assistant that can access external functions. The responses from these function calls will be appended to this dialogue. Please provide responses based on the information from these function calls."},
    {"role": "user", "content": "What are some examples of calorie-free, caffeine-free liquids that individuals with diabetes can consume when sick?"}
]

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    tool_choice="auto",
)

print(json.dumps(response.choices[0].message.model_dump()['tool_calls'], indent=2))


[
  {
    "id": "call_e7Kf1WyAthwLD8Mr5zXDFsJl",
    "function": {
      "arguments": "{\"location\":\"sick\"}",
      "name": "diabetes_patient"
    },
    "type": "function"
  }
]
